In [1]:
import torch
import torch_sparse
import numpy as np
from rgnn_at_scale.aggregation import (_sparse_top_k, soft_weighted_medoid, soft_weighted_medoid_k_neighborhood,
                                       weighted_dimwise_median, weighted_medoid, weighted_medoid_k_neighborhood)
from rgnn_at_scale.data import prep_graph

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
device = 0 if torch.cuda.is_available() else 'cpu'
temperature = 1e-3
device

'cpu'

In [4]:
A = torch.tensor([[0.5, 0.3, 0, 0.4],
                  [0.3, 0.2, 0, 0],
                  [0, 0, 0.9, 0.3],
                  [0.4, 0, 0.4, 0.4]],
                 requires_grad=True,
                 dtype=torch.float32).to(device)
x = torch.tensor([[-10, 10, 10],
                  [-1, 1, 1],
                  [0, 0, 0],
                  [10, -10, -10]],
                 requires_grad=True,
                 dtype=torch.float32).to(device)

A_sparse_tensor = torch_sparse.SparseTensor.from_dense(A).to(device)


In [6]:
medoids = soft_weighted_medoid_k_neighborhood(A_sparse_tensor,
                                                      x,
                                                      k=2,
                                                      temperature=temperature,
                                                      threshold_for_dense_if_cpu=0)
medoids

tensor([[-12.,  12.,  12.],
        [ -5.,   5.,   5.],
        [  0.,   0.,   0.],
        [ -6.,   6.,   6.]], grad_fn=<MulBackward0>)

In [8]:
graph = prep_graph("cora_ml", device)
attr, adj, labels = graph[:3]
A_sparse_tensor = torch_sparse.SparseTensor.from_torch_sparse_coo_tensor(adj).to(device)
x = attr
A_sparse_tensor, x

(SparseTensor(row=tensor([   0,    0,    0,  ..., 2808, 2809, 2809]),
              col=tensor([   0, 1579, 1581,  ..., 2808, 1399, 2809]),
              val=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
              size=(2810, 2810), nnz=18772, density=0.24%),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [9]:
 medoids = soft_weighted_medoid_k_neighborhood(A_sparse_tensor,
            x,
            k=2,
            temperature=temperature,
            threshold_for_dense_if_cpu=0)

In [10]:
medoids

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.1140],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [12]:
A_sparse_tensor = torch_sparse.SparseTensor.from_dense(torch.load("datasets/test_adj_matrix.pt"))
#x = torch_sparse.SparseTensor.from_dense(torch.load("datasets/test_attr_matrix.pt"))
x = torch.load("datasets/test_attr_matrix.pt")
A_sparse_tensor, x

(SparseTensor(row=tensor([   0,    0,    0,  ..., 2993, 2993, 2993]),
              col=tensor([1636, 1638, 2357,  ...,  200,  745, 1865]),
              val=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
              size=(2995, 2995), nnz=8416, density=0.09%),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [19]:
# number of nodes with no outgoing edges 
A_sparse_tensor.size(0) - A_sparse_tensor.sum(-1).nonzero().size(0)

349

In [20]:
medoids = soft_weighted_medoid_k_neighborhood(A_sparse_tensor,
            x,
            k=2,
            temperature=temperature,
            threshold_for_dense_if_cpu=0)

In [21]:
medoids

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0855],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [23]:
# this should have the same number of nonzeros
# or at least more of them, but it mustn't have less
medoids.size(0) - medoids.sum(-1).nonzero().size(0)

349

In [24]:
# now lets see if it works with a batched adj matrix
A_sparse_tensor = A_sparse_tensor[:512]

In [26]:
# calc number of nodes with no outgoing edges again
A_sparse_tensor.size(0) - A_sparse_tensor.sum(-1).nonzero().size(0)

33

In [30]:
medoids = soft_weighted_medoid_k_neighborhood(A_sparse_tensor,
            x,
            k=2,
            temperature=temperature,
            threshold_for_dense_if_cpu=0)

In [31]:
medoids

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [32]:
# check num zeros again
medoids.size(0) - medoids.sum(-1).nonzero().size(0)

33